In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

df =  pd.read_csv('flights_NY.csv')
df

In [ ]:
df.describe()

**В каких колонках есть пропущенные значения?**

In [70]:
for i in df:
    if df.isna()[i].any():
        print('В столбце', i, 'есть пропуски')

**Сколько строк, в которых есть хотя бы одно пропущенное значение?**

In [71]:
count, sk_ar = 0, []
for i in range(len(df)):
    if df.loc[i].isna().any():
        count += 1
        sk_ar.append(df.loc[i])
print(count, 'строк с пропусками')

 **Есть ли какая-то особенность в тех рейсах, в которых есть пропущенные значения?** 

In [ ]:
df_sk = pd.DataFrame(sk_ar)
df_sk.describe()

In [ ]:
if not df_sk['air_time'].notna().any():
    print('В любой строке с пропусками не указано время в полете')
if not df_sk['arr_delay'].notna().any():
    print('В любой строке с пропусками не указана задержка прибытия')
if df['dep_delay'].mean() * 2 < df_sk['dep_delay'].mean():
    print('В рейсах с пропусками задержка средняя задержка вылета сильно больше')

**Удалите строки, в которых есть хотя бы одно пропущенное значение из дальнейшего анализа.**

In [ ]:
df.dropna(inplace=True)
df

**Постройте в одних осях нормированные гистограммы времени задержки вылета и прилета.** 

In [ ]:
plt.hist(x = df['dep_delay'], density=True, label = 'Задержка вылета')
plt.hist(x = df['arr_delay'], density=True, label = 'Задержка прилета')
plt.legend()
plt.show()

**Ограничьте диапазон построения гистограмм, чтобы избавиться от выбросов, и опишите характер выбросов (количество и значения).**

In [ ]:
for x in ['arr_delay', 'dep_delay']:
    q75, q25 = np.percentile(df.loc[:,x],[75,25])
    intr_qr = q75 - q25

    up = q75 + (1.5*intr_qr)
    low = q25 - (1.5*intr_qr)
    
    print('Среднее для выбросов в меньшую сторону для', x, ':', round(df.loc[df[x] < low,x].mean()))
    #странно брать среднее для выбросов...
    print('Среднее для выбросов в большую сторону для', x, ':', round(df.loc[df[x] > up,x].mean()))
    df.loc[df[x] < low,x] = np.nan
    df.loc[df[x] > up,x] = np.nan
    print('В', x, df[x].isnull().sum(), 'выбросов')
df = df.dropna(axis = 0)
plt.hist(x = df['dep_delay'], density=True, label = 'Задержка вылета')
plt.hist(x = df['arr_delay'], density=True, label = 'Задержка прилета')
plt.legend()
plt.show()

**Есть ли другие особенности в полученных распределениях?**

In [ ]:
plt.boxplot(df['arr_delay'])
plt.show()

In [ ]:
plt.boxplot(df['dep_delay'])
plt.show()

Заметим, что распределения симметричны около нуля, напоминают нормальные

**Оцените среднее значение, медиану и величину стандартного отклонения для времени задержки вылета и времени задержки прилета**

In [ ]:
df['dep_delay'].describe()

In [ ]:
df['arr_delay'].describe()

**Отсортируйте авиакомпании по величине средней задержки вылета и приведите среднюю задержку вылета вместе с 95%-доверительным интервалом по каждой авиакомпании. Результат представьте в виде графика.**

In [ ]:
means = df.groupby('carrier').mean()['dep_delay'].sort_values()
airlines = [means[means == i].index[0] for i in means]
stds = df.groupby('carrier').std()['dep_delay']

scopes = pd.Series(dtype=str,name='scopes')
conf_iter = pd.Series(dtype=str,name='confidence_interval')
for i in airlines:
    n = df['carrier'].value_counts()[i]
    se = stds[i]/(n**0.5)
    sigma = stats.norm().isf(0.025) #(1-0.95)/2
    scopes.loc[i] = sigma*se*2
    conf_iter.loc[i] = [round(means[i] - sigma*se,2), round(means[i] + sigma*se,2)]
dd_a = pd.concat([means, conf_iter], axis=1)
dd_a

In [ ]:
plt.plot(dd_a['dep_delay'],'.-')

In [ ]:
data_dict = {}
data_dict['category'] = airlines
data_dict['lower'] = [i[0] for i in dd_a['confidence_interval']]
data_dict['upper'] = [i[1] for i in dd_a['confidence_interval']]
dataset = pd.DataFrame(data_dict)
for lower, upper, y in zip(dataset['lower'],dataset['upper'],range(len(dataset))):
    plt.plot((lower,upper),(y,y),'.-', color='green')
plt.yticks(range(len(dataset)),list(dataset['category']))
plt.show()

**Значимо ли различие в среднем времени задержки вылета для авиакомпаний American Airlines (AA) и Delta Airlines (DL)?На каком уровне значимости можно отвергнуть гипотезу о равенстве средних?**

Пусть $H_0$ - нет различия между средним временем задержки вылета для AA и DL, $\alpha = 0.05$. Используем двусторонний t-критерий 

In [ ]:
n = df['carrier'].value_counts()['AA']
t = (means['AA'] - means['DL']) * (n**0.5) / stds['AA']
print('t-value', t) #как-то неприятно...
print('p-value', stats.t.sf(abs(t), df=n-1)*2) #очень странно
#рассмотрим для случайной выборки из 400 полетов

In [ ]:
num = 400
s_means = df[df['carrier'] == 'AA'].sample(n=num, random_state=1).groupby('carrier').mean()['dep_delay']
s_stds = df[df['carrier'] == 'AA'].sample(n=num, random_state=1).groupby('carrier').std()['dep_delay']
t = (s_means['AA'] - means['DL']) * (num**0.5) / s_stds['AA']
p = stats.t.sf(abs(t), df=n-1)*2
p

Различия статистически не значимы. Для выборки из 400 человек достаточно $\alpha = 0.1 > p$ уровня значимости, чтобы считать среднее время задержки различным и отклонить нулевую гипотезу. 

**Сравните между собой аэропорты вылета (JFK, LGA, EWR) с точки зрения статистики задержек вылетов. Являются ли различия статистически значимыми?**

In [ ]:
num = 400

means = df.groupby('origin').mean()['dep_delay'].sort_values()
stds = df.groupby('origin').std()['dep_delay']
#JFK и LGA
s_means = df[df['origin'] == 'JFK'].sample(n=num, random_state=1).groupby('origin').mean()['dep_delay']
s_stds = df[df['origin'] == 'JFK'].sample(n=num, random_state=1).groupby('origin').std()['dep_delay']
t = (s_means['JFK'] - means['LGA']) * (num**0.5) / s_stds['JFK']
p_jl = stats.t.sf(abs(t), df=n-1)*2
#JFK и EWR
t = (s_means['JFK'] - means['EWR']) * (num**0.5) / s_stds['JFK']
p_je = stats.t.sf(abs(t), df=n-1)*2
#LGA и EWR
s_means = df[df['origin'] == 'LGA'].sample(n=num, random_state=1).groupby('origin').mean()['dep_delay']
s_stds = df[df['origin'] == 'LGA'].sample(n=num, random_state=1).groupby('origin').std()['dep_delay']
t = (s_means['LGA'] - means['EWR']) * (num**0.5) / s_stds['LGA']
p_le = stats.t.sf(abs(t), df=n-1)*2

for i,j in zip([p_jl, p_je, p_le], ['JFK и LGA', 'JFK и EWR', 'LGA и EWR']):
    print('Различия задержек вылетов', j, 'статитически', ''  if i < 0.05 else 'не', 'значимы')

**Каким распределением можно описать распределение времени задержки вылета в диапазоне, где время задержки вылета > 0?** 

In [ ]:
n = 400
dd = df[df['dep_delay'] > 0]['dep_delay'].sample(n)
dd.hist()
#напоминает распределение Пуассона...проверим, используя критерий Колмогорова-Смирнова

**Предложите общий вид распределения и оцените его параметры. На одном рисунке изобразите гистограмму и график плотности аппроксимирующего распределения.**

In [ ]:
import math

lmb = 9 #std
exp = [round(lmb**k * math.e**((-1)*lmb) / math.factorial(k),4)*270 for k in range(20)]
print(stats.kstest(rvs = dd, cdf = exp))
print('Не можем отвергнуть нулевую гипотезу, распределение действительно пуассоновское')

In [ ]:
import seaborn as sns

e = [i/270 for i in exp]
plt.hist(e)
sns.kdeplot(e)
plt.show()

**Для тех рейсов, для которых задержка вылета > 0, постройте в одних осях графики числа рейсов в месяц и среднего времени задержки в месяц.**

In [ ]:
dm = df[df['dep_delay'] > 0].groupby('month').mean()['dep_delay']
df = df.assign(useless = [1] * len(df))
nm = df[df['dep_delay'] > 0].groupby('month').sum()['useless']
del df['useless']
nm.rename('num_flights', inplace=True)
plt.plot(dm, label = 'Задержка вылета')
plt.plot(nm, label = 'Число рейсов')
plt.legend()
plt.show()

**Найдите коэффициент корреляции между полученными значениями.** 

In [ ]:
corr = round(np.corrcoef(dm, nm)[1][0],3)
print('Коэффициент корреляции', corr)

**Постройте точечную диаграмму (scatterplot), показывающую зависимость между полученными значениями (по оси Х отложите число рейсов в месяц). Нанесите на график линию регрессии. Выпишите уравнение регрессии.**

In [ ]:
plt.scatter(nm, dm)
 
coef = np.polyfit(nm, dm, 1)
poly1d_fn = np.poly1d(coef) 

plt.plot(nm, dm, 'yo', nm, poly1d_fn(nm), '--k')

plt.show()
x = dm.std()/nm.std()*corr
b = dm.mean() - nm.mean()*x
print('Уравнение линейной регреcсии: y = ', round(x,4), 'x + ', round(b,4), sep='')

**Постройте график среднего времени задержки в зависимости от часа вылета. На отдельном графике постройте долю рейсов, для которых задержка > 0, в зависимости от часа вылета. Опишите словами наблюдаемую картину.**


In [ ]:
def get_hour(s):
    s = str(int(s))
    if len(s) == 1:
        return 0
    elif (len(s) == 2) or (len(s) == 3):
        return int(s[0])
    else:
        if int(s[0:2]) == 24:
            return 0
        return int(s[0:2])
def dh_gr(d):
    d = d.assign(hour_dep = d['dep_time'].apply(get_hour))
    dh = d.groupby('hour_dep').mean()['dep_delay']   
    plt.plot(dh)
    plt.show()
    #return dh
dh_gr(df[df['dep_delay'] > 0])#-dh_gr(df)
dh_gr(df)

До четырех утра самолеты не отправляются раньше времени, в 4 утра максимальное отклонение от времени отправления, в 5 утра - минимальное, после 8 часов вечера отклонение от времени отправления растет

**Предложите способ разделить авиакомпании на пунктуальные и непунктуальные. Какие авиакомпании в какую группу попадают? Будут ли различаться группы в зависимости от дальности перелета?**


Будем считать непунктуальными компании, среднее время задержек вылетов больше 3 квартили и размах 95% доверительного интервала больше 1. Группы не будут различаться в зависимости от дальности перелета

In [ ]:
m = df['dep_delay'].apply(abs).quantile(0.75)
print('Итого непунктуальными будем считать: ', end='')
punctual_airlines = []
for i in airlines:
    if (means[i] > m) and (scopes[i] > 1):
        punctual_airlines.append(i)
        print(i,end=' ')
        
#df = df.assign(punctual = [i in punctual_airlines for i in df['carrier']])
#df

**Предложите модель, дающую прогноз средней задержки вылета по каждому аэропорту на следующий день. Какие признаки вы будете использовать, на каких данных будете обучать (настраивать) модель, на каких тестировать? Оцените точность вашей модели. В частности, сделайте прогноз на 31 декабря 2013 года и сравните с фактическим значением (исключите эту дату из обучающей и тестовой выборки).**


In [ ]:
from sklearn. model_selection import train_test_split
from sklearn. linear_model import LogisticRegression
from sklearn import metrics
df = df.assign(hour_dep = d['dep_time']

**Приведите еще хотя бы один интересный факт из датасета, раскрывающий или характеризующий особенности в возникновении/распределении задержек авиарейсов.**


In [ ]:
plt.figure(figsize=(12,10), dpi= 80)
sns.heatmap(df.corr(), xticklabels=df.corr().columns, yticklabels=df.corr().columns, cmap='RdYlGn', center=0, annot=True)


plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
dd = df.groupby('day').mean()['dep_delay']   
plt.plot(dd)

Заметим, что задержка отправления существенно зависит от дня вылета.

In [ ]:
for i in ['month','day','dep_time','dep_delay','arr_time','arr_delay','flight','air_time','distance']:
    for j in ['month','day','dep_time','dep_delay','arr_time','arr_delay','flight','air_time','distance']:
        if 1 >round(np.corrcoef(df[j], df[i])[1][0],3) > 0.2:
            print(i,j, round(np.corrcoef(df[j], df[i])[1][0],3))

Видим, высокий уровень корреляции между задержкой вылета и задержкой прибытия, что неудивительно

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class NeuralNetwork():
    def __init__(self, ):
        self.inputSize = 3
        self.outputSize = 1
        self.hiddenSize = 3

        self.W1 = np.random.rand(self.inputSize, self.hiddenSize)
        self.W2 = np.random.rand(self.hiddenSize, self.outputSize)
        self.limit = 0.5

        self.error_list = []
        self.true_positives = 0
        self.false_positives = 0
        self.true_negatives = 0
        self.false_negatives = 0

    def forward(self, X):
        self.z = np.matmul(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = np.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3)
        return o

    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))

    def sigmoidPrime(self, s):
        return s * (1 - s)

    def backward(self, X, y, o):
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        self.z2_error = np.matmul(self.o_delta, np.matrix.transpose(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += np.matmul(np.matrix.transpose(X), self.z2_delta)
        self.W2 += np.matmul(np.matrix.transpose(self.z2), self.o_delta)

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            o = self.forward(X)
            self.backward(X, y, o)
            self.error_list.append(np.abs(self.o_error).mean())

    def predict(self, x_predicted):
        return self.forward(x_predicted).item()

    def view_error_development(self):
        plt.plot(range(len(self.error_list)), self.error_list)
        plt.title('Mean Sum Squared Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

    def test_evaluation(self, input_test, output_test):
        for i, test_element in enumerate(input_test):
            if self.predict(test_element) > self.limit and output_test[i] == 1:
                self.true_positives += 1
            if self.predict(test_element) < self.limit and output_test[i] == 1:
                self.false_negatives += 1
            if self.predict(test_element) > self.limit and output_test[i] == 0:
                self.false_positives += 1
            if self.predict(test_element) < self.limit and output_test[i] == 0:
                self.true_negatives += 1
        print('True positives: ', self.true_positives, '\nTrue negatives: ', self.true_negatives,
              '\nFalse positives: ', self.false_positives, '\nFalse negatives: ', self.false_negatives,
              '\nAccuracy: ', (self.true_positives + self.true_negatives) /
              (self.true_positives + self.true_negatives + self.false_positives + self.false_negatives))